In [1]:
# importing modules
import pandas as pd
import os
from datetime import date
from datetime import datetime
import time
import pytz
from ast import Name
from google.colab import drive

In [ ]:
# Mount google drive
drive.mount('/content/drive')

# Folder path containing the Excel files
folder_path = '/content/blue'

# Initialize an empty DataFrame to store the merged data
merged_data = pd.DataFrame()

# Iterate over each file in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith(".xlsx") or file_name.endswith(".xls"):

        # Construct the full file path
        file_path = os.path.join(folder_path, file_name)

        # Read the Excel file
        df = pd.read_excel(file_path, header = 1)

        # Add a "filename" column with a specific value based on the file title
        df["Filename"] = file_name

        # Rename columns
        df.rename(columns={'Resource Type': 'EMResource Region',
                       'Resource': 'Facility Name',}, inplace=True)


 ############### time related coding ###############

        # saving or exporting files to current date
        current_date = str(time.strftime("%Y-%m-%d %H:%M:%S"))

        # Add "date event started" and "date event ended" columns
        df["Date Event Started"] = '10/30/2023 11:01' ##### hardcoded, need to change #####
        df["Date Event Ended"] = ''

        # Get the current UTC time
        utc_time = datetime.utcnow()

        # Define the time zone for Colorado (Mountain Time Zone)
        colorado_tz = pytz.timezone('America/Denver')

        # Convert the UTC time to Colorado time zone
        colorado_time = utc_time.replace(tzinfo=pytz.utc).astimezone(colorado_tz)

        # Format the time in Colorado's timezone
        df['Date Collected'] = colorado_time.strftime('%Y-%m-%d %H:%M:%S %Z')
        df['Time'] = colorado_time.strftime('%H:%M:%S %Z')

############## merging files together ###############

        # Append the processed data to the merged DataFrame
        merged_data = merged_data.append(df)

############### adding license ID to file ###############

# read file containing the license ID information for each facility
path = '/content/facilities.xlsx'
path1 = pd.read_excel (path)

# drop unneeded columns in the datafile
drop_col = ['Type', 'EMResource ID', 'AHA ID', 'External ID', 'Website', 'Contact',
       'Phone 1', 'Phone 2', 'Fax', 'Email', 'Notes']

idno = path1.drop(columns = drop_col)

# Rename columns
idno.rename(columns={'Resource Name' : 'Facility Name',
                   'License #  (State will populate)': 'License ID'}, inplace=True)

# Merging license ID # data to merged data
merge_column = 'Facility Name'
result_df = merged_data.merge (idno, on=merge_column)

# Save the merged DataFrame to a new Excel file
# output_file_path = os.path.join(folder_path, 'merged_license_%s.xlsx' % current_date)
output_file_path = '/content/merged_license.xlsx'
result_df.to_excel(output_file_path, index=False)

Things to add:
- pull directly from EMR (Danielle)
- can you add FACID data to file?  Hospital names aren't the same, linking to Tableau might be difficult (Himaja) single identifier
- pull out address from SQL for map visualizations (MedOPs Team Angie)
- can you connect this to google drive
- missing data between original and merged file
- date collection to be the same as file name or based on time of collection (every 15 minutes)
- rename "resource type" and "retac" to "EMResource Region"
- output file named the same way in the column
- google drive that can be shared by everyone
- Merged output file should always be the same file name

07-17-2023 - first phase features : reads each excel file in a folder at once, add a column called filename (gives the name of the file name, so we'd know which document it is coming from), add columns for when the date event started and ended, rename some columns (e.g resource type ---> RETAC, resource --> facility name...etc), renaming the values in the column for filename to specific timepoints collected, then merging them all at once and saving the output file for use.

07-21-2023 - first phase update : added automated time rather than specific time for each document, added facility ID, address, county, latitude, longitude...columns

08-03-2023 - added the ability to connect to google drive and work based on google drive, rename "resource type" and "retac" to "EMResource Region"
